## CNN :

In [9]:
import tensorflow as tf
from keras import layers
from keras.datasets import cifar10
import matplotlib.pyplot as plt
from keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping,
)

from keras import Model
from keras.models import Sequential

from keras.utils import to_categorical
from keras.applications import ResNet50
from keras.optimizers import AdamW, RMSprop

In [10]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [11]:
inputs = tf.keras.Input(shape=(32, 32, 3))

# Block 1
x = layers.Conv2D(64, (3, 3), padding="same", activation="relu")(inputs)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Dropout(0.25)(x)

# Block 2
x = layers.Conv2D(128, (3, 3), padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Dropout(0.25)(x)

# Block 3
x = layers.Conv2D(256, (3, 3), padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Dropout(0.4)(x)

# Classifier
x = layers.Flatten()(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.45)(x)

outputs = layers.Dense(10, activation="softmax")(x)

model = Model(inputs, outputs)

In [12]:
optimizer = AdamW(learning_rate=0.0005)

model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)

lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=3, min_lr=1e-5)

early_stop = EarlyStopping(
    patience=3,
    restore_best_weights=True,
    monitor="val_accuracy",
    min_delta=0.001,
)

## MODEL TRAINING

In [20]:
history = model.fit(
    train_images,
    train_labels,
    validation_split=0.2,
    epochs=25,
    batch_size=128,
    callbacks=[lr_scheduler, early_stop],
    verbose=1,
)

Epoch 1/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8845 - loss: 0.3292 - val_accuracy: 0.8227 - val_loss: 0.5322 - learning_rate: 6.2500e-05
Epoch 2/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.8862 - loss: 0.3274 - val_accuracy: 0.8169 - val_loss: 0.5493 - learning_rate: 6.2500e-05
Epoch 3/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.8867 - loss: 0.3239 - val_accuracy: 0.8168 - val_loss: 0.5549 - learning_rate: 6.2500e-05
Epoch 4/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8919 - loss: 0.3084 - val_accuracy: 0.8218 - val_loss: 0.5426 - learning_rate: 6.2500e-05


In [24]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)

print(f"CNN Test Accuracy: {test_acc:.4f}")

CNN Test Accuracy: 0.8196


##  ResNet50 using Transfer Learning

In [25]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = tf.keras.applications.resnet50.preprocess_input(x_train)
x_test = tf.keras.applications.resnet50.preprocess_input(x_test)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print("Train data shape:", x_train.shape)
print("Test data shape:", x_test.shape)

Train data shape: (50000, 32, 32, 3)
Test data shape: (10000, 32, 32, 3)


In [26]:
base_model = ResNet50(
    include_top=False, weights="imagenet"
)

# We freeze all but the last layers, so only the last layer's weights are trainable
for layer in base_model.layers[:143]:
    layer.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [27]:
model = Sequential(
    [
        layers.Lambda(lambda image: tf.image.resize(image, (224, 224))),
        base_model,
        layers.Flatten(),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.BatchNormalization(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.BatchNormalization(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.BatchNormalization(),
        layers.Dense(10, activation="softmax"),
    ]
)

model.compile(optimizer=RMSprop(0.00002), loss="categorical_crossentropy", metrics=["accuracy"])

lr_schedule = (
    ReduceLROnPlateau(
        patience=1, factor=0.5, min_lr=1e-6, monitor="val_accuracy", verbose=1
    ),
)

es = EarlyStopping(
    monitor="val_accuracy", mode="max", patience=2, restore_best_weights=True
)

In [28]:
history = model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=10,
    callbacks=[es, lr_schedule],
    validation_split=0.2,
    verbose=1,
)

Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 207s 145ms/step - accuracy: 0.2531 - loss: 2.2611 - val_accuracy: 0.8462 - val_loss: 0.7739 - learning_rate: 2.0000e-05
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 196s 150ms/step - accuracy: 0.5457 - loss: 1.3809 - val_accuracy: 0.8920 - val_loss: 0.5272 - learning_rate: 2.0000e-05
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 202s 150ms/step - accuracy: 0.6910 - loss: 1.0338 - val_accuracy: 0.9164 - val_loss: 0.3930 - learning_rate: 2.0000e-05
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 202s 150ms/step - accuracy: 0.7727 - loss: 0.8347 - val_accuracy: 0.9322 - val_loss: 0.3138 - learning_rate: 2.0000e-05
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 202s 150ms/step - accuracy: 0.8273 - loss: 0.6771 - val_accuracy: 0.9388 - val_loss: 0.2659 - learning_rate: 2.0000e-05
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 171s 137ms/step - accuracy: 0.8689 - loss: 0.5532 - val_accuracy: 0.9410 - val_loss: 0.2350 - learning_rate: 2.0000e-05
Epoch 7/10
1250/1250 ━━━━━━━

In [29]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"\nFinal Test Accuracy: {test_accuracy:.4f}")


Final Test Accuracy: 0.9469


## Performance Analysis
CNN Validation Accuracy: ~81%

ResNet50 Validation Accuracy: ~94%

## Training Configuration
Component	Value
Optimizer	AdamW (CNN), RMSprop (ResNet)
Loss Function	categorical_crossentropy
Batch Size	128
Epochs	25 (Early stopping used)
Regularization	Dropout + BatchNormalization
Learning Rate Schedule	ReduceLROnPlateau with patience and minimum LR set
Early Stopping	Based on val_accuracy with patience = 3


## Observations
The CNN model performed moderately well (~81% accuracy), showing it's suitable for baseline image classification on CIFAR-10.

The ResNet50 model, leveraging transfer learning, significantly improved validation accuracy (~94%), demonstrating the strength of deep pretrained architectures.

Batch normalization and dropout were used effectively to combat overfitting.

Early stopping ensured efficient training without unnecessary over-epoching.